**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, \
                         Reshape, BatchNormalization, Flatten
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function ***act*** represents basically the policy. Indeed, according to the current state s, act returns the action the ```Agent``` must use. Here, the function ***act*** is divided in two steps. During the training step, the function ***act*** is a trade-off between exploration and exploitation. The goal of epsilon is precisely to explore more state: the ```Agent``` is going to use a random action with probability epsilon. Otherwise, the ```Agent``` uses the learned policy (***learned_act***). Therefore, the goal of epsilon is to encourage the exploration during the training period.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, 
                                 grid_size * self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, 
                       interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :]= -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0: # Going up (or down on the image)
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1: # Going up on the image
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2: # Going right on the image
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # Going left on the image
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                               axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, 
                                 self.grid_size * self.scale, 3))

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)), 
                               axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 21 # set small when debugging
epochs_test = 15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

\begin{itemize}
    \item The position array represents the position of the rat on the grid. One particular aspect is the border: the 2 first / last rows and columns are considered as walls and cannot be access. Therefore, the value of position for these cases are defined as -1. The position where is the rat is valued at 1.
    \item The board array represents the different rewards that the rat can obtain for every cases of the grid. Typically, the board value is equal to 0.5 where there is cheese, equal to -1 where there is poison and 0 otherwise.
\end{itemize}


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        """Return a rabdom action."""
        
        action = np.random.randint(0, 4, 1)[0]
        
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix='', display=True, save=True):
    """Visualise the moves of the rat"""
    
    # Number of won games
    score = 0
        
    for e in range(epochs):

        # Initialisation of win and lose
        win = 0
        lose = 0
        
        # This assumes that the games will end
        game_over = False

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        while not game_over:
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
#             loss = agent.reinforce(prev_state, state, action, reward, game_over)
            
        # Save as a mp4
        if save:
            env.draw(prefix + str(e))

        # Update stats
        score = score + win-lose
        
        if display:
            print("Win/lose count {}/{}. Average score ({})"
                  .format(win, lose, score / epochs))
        
    if display:
        print('Final score: ' + str(score/epochs))
    
    return score

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='./Results/random')
HTML(display_videos('./Results/random0.mp4'))

Win/lose count 10.0/19.0. Average score (-0.6)
Win/lose count 6.5/22.0. Average score (-1.6333333333333333)
Win/lose count 7.0/12.0. Average score (-1.9666666666666666)
Win/lose count 7.0/10.0. Average score (-2.1666666666666665)
Win/lose count 5.0/13.0. Average score (-2.7)
Win/lose count 7.0/18.0. Average score (-3.433333333333333)
Win/lose count 11.5/16.0. Average score (-3.7333333333333334)
Win/lose count 8.0/14.0. Average score (-4.133333333333334)
Win/lose count 6.0/6.0. Average score (-4.133333333333334)
Win/lose count 9.5/16.0. Average score (-4.566666666666666)
Win/lose count 10.0/11.0. Average score (-4.633333333333334)
Win/lose count 10.5/14.0. Average score (-4.866666666666666)
Win/lose count 9.0/16.0. Average score (-5.333333333333333)
Win/lose count 9.5/8.0. Average score (-5.233333333333333)
Win/lose count 7.5/11.0. Average score (-5.466666666666667)
Final score: -5.466666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




\begin{itemize}
    \item The first result represents the Bellman equation. Let's prove it.
    We denote by $(S_t)$ and $(A_t)$ the series of random variables representing the states and action over time. Therefore, if we take up the notations of the slides of the course, it has:
    \begin{align*}
        Q^{\pi}(s_t, a_t) &= \mathbb{E}_{\pi} \left[ \sum_{k=0}^{+\infty} \gamma^k * r(s_{t + k}, a_{t + k}) ~ \big| ~ S_t=s_t, A_t=a_t \right] \\
        &= r(s_t, a_t) + \gamma \mathbb{E}_{\pi} \left[ \sum_{k=0}^{+\infty} \gamma^k * r(s_{(t + 1) + k}, a_{(t + 1) + k}) ~ \big| ~ S_t=s_t, A_t=a_t \right] \\
        &= r(s_t, a_t) + \gamma \mathbb{E}_{\pi} \left[ \sum_{s_{t+1}, a_{t+1}} \mathbf{1}_{ \{ S_{t+1} = s_{t+1}, A_{t+1} = a_{t+1} \} } \underbrace{\mathbb{E}_{\pi} \left[ \sum_{k=0}^{+\infty} \gamma^k * r(s_{(t + 1) + k}, a_{(t + 1) + k}) ~ \big| ~ S_{t+1}=s_{t+1}, A_{t+1}=a_{t+1} \right]}_{ = Q^{\pi}(s_{t+1}, a_{t+1})} ~ \big| ~ S_t=s_t, A_t=a_t \right] \\
        &= r(s_t, a_t) + \gamma \sum_{s_{t+1}, a_{t+1}} \mathbb{P}^{\pi}\left(S_{t+1}=s_{t+1}, A_{t+1}=a_{t+1} ~ | ~ S_t=s_t, A_t=a_t \right) Q^{\pi}(s_{t+1}, a_{t+1}) \\
        &= r(a_t, s_t) + \gamma \mathbb{E}_{(s_{t+1}, a_{t+1}) \sim p(.,. | s_t, a_t)} \left[ Q^{\pi}(s_{t+1}, a_{t+1}) \right]
    \end{align*}
    Moreover, these results are true for every $t$. So, it obtains the result.   
    \item This second equation corresponds to the Optimal Bellman Equation. Let's prove it: If it uses again the same equations with $Q^*$, it has:
    \begin{align*}
    Q^{*}(s_t, a_t) &= r(s_t, a_t) + \gamma \sum_{s_{t+1}, a_{t+1}} \mathbb{P}^{\pi^*}\left(S_{t+1}=s_{t+1}, A_{t+1}=a_{t+1} ~ | ~ S_t=s_t, A_t=a_t \right) Q^{*}(s_{t+1}, a_{t+1}) \\
    &= r(s_t, a_t) + \gamma \sum_{s_{t+1}, a_{t+1}} \mathbb{P}\left(S_{t+1}=s_{t+1} ~ | ~ S_t=s_t, A_t=a_t \right) \pi^*(a_{t+1} ~ | ~ S_{t+1} = s_{t+1}) Q^{*}(s_{t+1}, a_{t+1})
    \end{align*}
    But, by definition of the optimal policy, 
    \begin{equation*}
        \pi^*(a_{t+1} ~ | ~ S_{t+1} = s_{t+1}) = \left\{ \begin{array}{ cl}
                                                            \frac{1}{|\text{argmax}_{a} Q^{*}(s_{t+1}, a)|}, & \text{if} ~ a_{t+1} \in \text{argmax}_{a} Q^{*}(s_{t+1}, a) \\
                                                            0, & \text{otherwise}
                                                          \end{array}
                                                  \right.
   \end{equation*}
  Therefore:
   \begin{align*}
    Q^{*}(s_t, a_t) &= r(s_t, a_t) + \gamma \sum_{s_{t+1}} \mathbb{P}\left(S_{t+1}=s_{t+1} ~ | ~ S_t=s_t, A_t=a_t \right) max_{a_{t+1}} Q^{*}(s_{t+1}, a_{t+1}) \\
    &= r(a_t, s_t) + \gamma \mathbb{E}_{s_{t+1} \sim p(. | s_t, \pi^*(.|s_t))} \left[ \max_{a_{t+1}} Q^{*}(s_{t+1}, a_{t+1}) \right]
    \end{align*}
    And as before, the result does not depend on the time $t$. It obtains theferore the result.
    \item So,now let suppose that we succeed to parametrised Q by a parameter $\theta$. So, if it is looking for the the optimal policy $\pi^*$, it corresponds to find $\theta$ such that the previous equation on Q is respected. So, we would like to find theta such the two terms of the previous equation are as close as possible. Therefore, we can compute:
    \begin{align*}
        \mathcal{L}_1(\theta) &= \Vert E_{s' \sim \pi^*(.|s,a)} \left[ r + \gamma \max_{a'}Q(s',a',\theta) \right] - Q(s, a, \theta) \Vert^2 \\
        &= \Vert E_{s' \sim \pi^*(.|s,a)} \left[ r + \gamma \max_{a'}Q(s',a',\theta) - Q(s, a, \theta) \right] \Vert^2 \\
        &\leq E_{s' \sim \pi^*(.|s,a)} \left[ \Vert r + \gamma \max_{a'}Q(s',a',\theta) - Q(s, a, \theta) \Vert^2 \right], ~~~ \text{by Jensen inequality} \\
        &\leq \mathcal{L}(\theta)
    \end{align*}
    So $\mathcal{L}(\theta)$ can be used as pausible objective.
\end{itemize}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        """Store (s_t, a_t, s_t+1, r_t, game_over_t)."""
        
        # Current length of the memory
        n = len(self.memory)
        
        # Append the new state
        if n < self.max_memory:
            self.memory.append(m)
        else:
            # Replace a random state
            idx = np.random.choice(np.arange(n))
            self.memory[idx] = m

    def random_access(self):
        """Extract a random quintuplet (s_t, a_t, s_t+1, r_t, game_over_t) in memory."""
        
        # Current length of the memory
        n = len(self.memory)
        
        # Choose a random index
        idx = np.random.choice(np.arange(n))
        quintuplet = self.memory[idx]
        
        return quintuplet

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix='', display=True, save=True):
    """Train the agent on env for the given number of epoch."""
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win-lose
        
        if display:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e + 1, epoch, loss, win, lose, win-lose))
        
        if save:
            agent.save(name_weights=prefix + 'model.h5', name_model=prefix + 'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        """Initialisation of Deep Q Network."""
        super().__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        # Grid size
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        """Choose the action according the current model."""
        
        # Reshape the state
        s_t = s.reshape(1, 5, 5, self.n_state)
        
        # Choose the maximising action
        a_t = np.argmax(self.model.predict(s_t))
        if a_t < 0:
            print(a_t)
        
        return a_t

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        """Two steps: first memorize the states, second learn from the pool."""

        # First Step
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # Second Step
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state)) #5, self.n_state))
        target_q = np.zeros((self.batch_size, 2), dtype=np.dtype(float, int))
        
        for i in range(self.batch_size):
            
            # Sample a random transition
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            # Add this state in input states
            input_states[i, :, :, :] = s_
            
            if game_over_:
                target_q[i, :] = r_
            else:
                n_s_ = n_s_.reshape(1, 5, 5, self.n_state)
                target_q[i, :] = [r_ + self.discount * np.max(self.model.predict(n_s_)), a_]
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -5, 5)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        
        self.model.save_weights(name_weights, overwrite=True)
        
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self, name_weights='model.h5', name_model='model.json'):
        
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
            
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

        
    # Custom loss
    def my_loss(self, data, y_pred):
        """Compute the mse between y_j and Q(s_j, a_j)."""

        # Length
        n = np.shape(y_pred)[0]

        # Extract the corresponding y_pred
        idx = tf.reshape(K.cast(data[:, 1], tf.int32), (-1, 1))
        index = tf.concat([tf.reshape(tf.range(0, self.batch_size), (-1, 1)), idx], axis=1)
        y_pred_2 = tf.gather_nd(y_pred, index)
        y_true = data[:, 0]

        return tf.losses.mean_squared_error(y_true, y_pred_2)

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state, )))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), self.my_loss)
        self.model = model
        
#         print(model.summary())        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1, epsilon=0.1, memory_size=1000, batch_size=128)
train(agent, env, epochs_train, prefix='./Results/fc_train')
HTML(display_videos('./Results/fc_train10.mp4'))

Epoch 001/021 | Loss 0.0112 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/021 | Loss 0.0395 | Win/lose count 4.5/5.0 (-0.5)
Epoch 003/021 | Loss 0.0241 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/021 | Loss 0.0283 | Win/lose count 1.0/2.0 (-1.0)
Epoch 005/021 | Loss 0.0189 | Win/lose count 6.5/4.0 (2.5)
Epoch 006/021 | Loss 0.0279 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/021 | Loss 0.0224 | Win/lose count 2.5/2.0 (0.5)
Epoch 008/021 | Loss 0.0142 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/021 | Loss 0.0127 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/021 | Loss 0.0163 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/021 | Loss 0.0182 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/021 | Loss 0.0198 | Win/lose count 9.0/2.0 (7.0)
Epoch 013/021 | Loss 0.0130 | Win/lose count 7.0/3.0 (4.0)
Epoch 014/021 | Loss 0.0215 | Win/lose count 5.5/1.0 (4.5)
Epoch 015/021 | Loss 0.0187 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/021 | Loss 0.0351 | Win/lose count 4.0/5.0 (-1.0)
Epoch 017/021 | Loss 0.0300 | Win/lose count 9.5/2.

In [16]:
def GridSearch(dqn, train_function=train, env=env):
    """Do a gridsearch on the following parameters: batch_size, memory_size, lr.
       """
    
    # Possible values of the optimising parameters
    batch_size_l = [64]
    memory_size_l = [1000]
    lr_l = [1, 0.1, 0.01, 0.001]
    
    # Initialisation of best parameters
    best_parameters = [0, 0, 0]
    max_score = -float("inf")
    i = 0
    
    # Gridsearch
    for batch_size in batch_size_l:
        for memory_size in memory_size_l:
            for lr in lr_l:
                
                # Definition of the agent
                agent = dqn(batch_size, memory_size, lr)
                
                # Training of the agent
                train_function(agent, env, epochs_train, prefix='./Results/fc_train_' + str(i) + "_",
                               display=False, save=False)
                
                # Testing of the agent
                score_test = test(agent, env, epochs_test,
                                  prefix='./Results/fc_test_' + str(i),
                                  display=False, save=False)
                
                # Update of best parameters
                if max_score < score_test:
                    max_score = score_test
                    best_parameters = [batch_size, memory_size, lr]                
                    print("New best score:", score_test)
                    print("Best parameters:", best_parameters)
                # Update of i
                i += 1
                n = len(batch_size_l) * len(memory_size_l) * len(lr_l)
                print("Step done: ", i / n * 100)
    
    # Display best parameters
    print(best_parameters)

In [ ]:
# Definition of the environment
env = Environment(grid_size=size, max_time=T, temperature=0.3)

# Definition of the agent
dqn = lambda batch_size, memory_size, lr: DQN_CNN(size, lr=lr, epsilon=0.1,
                                                  memory_size=memory_size,
                                                  batch_size=batch_size)

# Compute the GridSearch
GridSearch(dqn, env=env)

For obtaining the best results, I implemented a GridSearch over different values of the following parameters:
\begin{itemize}
    \item batch_size
    \item memory_size
    \item learning_rate
\end{itemize}
It seems that the two first parameters do not have a big impact for the environment selected. However, it seems that a big learning rate is required.

Also, we implemented the loss in the function "my_loss" which correspond to:
\begin{equation}
    \sum_{i=1}^N (Q(s_i, s_a) - y_i)^2, ~~ \text{with} ~~ y_i = \left\{\begin{array}{cl}
                                                                           r_i & \text{if} ~ \text{game_over} = True \\
                                                                           r_i + \max_{a'} Q({s'}_i, a') & \text{otherwise}
                                                                       \end{array}
                                                                \right.
\end{equation}

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [18]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        
        ## CNN
        model = Sequential()
        model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(5, 5, self.n_state, )))
        model.add(Conv2D(16, (3, 3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), self.my_loss)
        self.model = model

In [19]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1, epsilon=0.1, memory_size=1000, batch_size=128)
train(agent,env,epochs_train,prefix='./Results/cnn_train')
HTML(display_videos('./Results/cnn_train10.mp4'))

Epoch 001/021 | Loss 0.0390 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/021 | Loss 0.0572 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/021 | Loss 0.0604 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/021 | Loss 0.0165 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/021 | Loss 0.0126 | Win/lose count 2.5/6.0 (-3.5)
Epoch 006/021 | Loss 0.0129 | Win/lose count 1.0/4.0 (-3.0)
Epoch 007/021 | Loss 0.0187 | Win/lose count 3.5/1.0 (2.5)
Epoch 008/021 | Loss 0.0166 | Win/lose count 6.5/5.0 (1.5)
Epoch 009/021 | Loss 0.0229 | Win/lose count 6.0/4.0 (2.0)
Epoch 010/021 | Loss 0.0130 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/021 | Loss 0.0360 | Win/lose count 5.0/1.0 (4.0)
Epoch 012/021 | Loss 0.0171 | Win/lose count 5.5/3.0 (2.5)
Epoch 013/021 | Loss 0.0170 | Win/lose count 5.5/2.0 (3.5)
Epoch 014/021 | Loss 0.0159 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/021 | Loss 0.0190 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/021 | Loss 0.0221 | Win/lose count 7.0/5.0 (2.0)
Epoch 017/021 | Loss 0.0235 | Win/lose count 2.5/4.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.1)
agent_cnn = DQN_CNN(size, lr=1, epsilon=0.1, memory_size=1000, batch_size=128)
agent_cnn.load(name_weights='./Results/cnn_trainmodel.h5',
               name_model='./Results/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1, epsilon=0.1, memory_size=1000, batch_size=128)
agent_cnn.load(name_weights='./Results/fc_trainmodel.h5',
               name_model='./Results/fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='./Results/cnn_test')
print('\nTest of the FC')
test(agent_fc, env, epochs_test, prefix='./Results/fc_test');

Test of the CNN
Win/lose count 3.0/3.0. Average score (0.0)
Win/lose count 3.0/0. Average score (0.2)
Win/lose count 2.0/2.0. Average score (0.2)
Win/lose count 2.5/2.0. Average score (0.23333333333333334)
Win/lose count 2.5/1.0. Average score (0.3333333333333333)
Win/lose count 2.5/1.0. Average score (0.43333333333333335)
Win/lose count 2.0/4.0. Average score (0.3)
Win/lose count 2.5/1.0. Average score (0.4)
Win/lose count 2.5/1.0. Average score (0.5)
Win/lose count 3.0/1.0. Average score (0.6333333333333333)
Win/lose count 1.0/1.0. Average score (0.6333333333333333)
Win/lose count 3.0/3.0. Average score (0.6333333333333333)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 3.5/1.0. Average score (0.8333333333333334)
Win/lose count 3.0/0. Average score (1.0333333333333334)
Final score: 1.0333333333333334

Test of the FC
Win/lose count 2.0/0. Average score (0.13333333333333333)
Win/lose count 1.0/1.0. Average score (0.13333333333333333)
Win/lose count 2.0/1.0. Ave

In [21]:
HTML(display_videos('./Results/cnn_test10.mp4'))

In [22]:
HTML(display_videos('./Results/fc_test10.mp4'))

For optimised parameters, we observed:
\begin{itemize}
    \item The performance of the DQN with Convolutionnal networks performs in average better than the Fully Connected one.
    \item Also, we can observe that the agent get easily blocked in the corners and only explore one border There is a lack of exploration here.
    \item When the temperature increases, the score of the two agents increase also a lot. Indeed, there is much more food in the environment so it is easy for the agent to increase its score.
\end{itemize}

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [25]:
def train_explore(agent, env, epoch, prefix='', display=True, save=True):
    """Train the agent to explore env for the given number of epoch."""
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Definition of epsilon
        epsilon = 1 / (1 + e)**2
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win-lose
        
        if display:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e + 1, epoch, loss, win, lose, win-lose))
        
        if save:
            agent.save(name_weights=prefix + 'model.h5', name_model=prefix + 'model.json')

            
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, train_binary=True):
        super().__init__(grid_size=grid_size, max_time=max_time,
                         temperature=temperature)
        self.train_binary = train_binary
        
    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :]= -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0: # Going up (or down on the image)
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1: # Going up on the image
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2: # Going right on the image
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # Going left on the image
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # Update of the reward
        reward = 0
        if self.train_binary:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                               axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
    
        # Definition of board
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, 
                                 self.grid_size * self.scale, 3))

        malus[bonus > 0] = 0

        self.board = bonus + malus
        
        # Definition of malus_position
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        # Definition of position
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                               axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [ ]:
# Definition of the environment
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)

# Definition of the agent
dqn = lambda batch_size, memory_size, lr: DQN_CNN(size, lr=lr, epsilon=0.1,
                                                  memory_size=memory_size,
                                                  batch_size=batch_size, n_state=3)

# Compute the GridSearch
GridSearch(dqn, train_function=train_explore, env=env)

In [27]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1, memory_size=1000, batch_size=128, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 001/021 | Loss 0.0510 | Win/lose count 5.0/33.700000000000124 (-28.700000000000124)
Epoch 002/021 | Loss 0.0472 | Win/lose count 10.0/24.20000000000007 (-14.20000000000007)
Epoch 003/021 | Loss 0.0386 | Win/lose count 11.0/18.70000000000001 (-7.70000000000001)
Epoch 004/021 | Loss 0.0341 | Win/lose count 11.0/18.59999999999998 (-7.59999999999998)
Epoch 005/021 | Loss 0.0354 | Win/lose count 14.5/18.700000000000014 (-4.2000000000000135)
Epoch 006/021 | Loss 0.0370 | Win/lose count 7.5/19.4 (-11.899999999999999)
Epoch 007/021 | Loss 0.0354 | Win/lose count 16.5/14.499999999999972 (2.0000000000000284)
Epoch 008/021 | Loss 0.0419 | Win/lose count 15.5/15.999999999999966 (-0.49999999999996625)
Epoch 009/021 | Loss 0.0323 | Win/lose count 9.5/16.29999999999996 (-6.799999999999962)
Epoch 010/021 | Loss 0.0358 | Win/lose count 6.5/18.599999999999994 (-12.099999999999994)
Epoch 011/021 | Loss 0.0414 | Win/lose count 5.5/17.799999999999983 (-12.299999999999983)
Epoch 012/021 | Loss 0.0295 

In [29]:
# Definition of the environment without training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, train_binary=False)

# Evaluation
test(agent, env, epochs_test,prefix='./Results/cnn_test_explore')
HTML(display_videos('./Results/cnn_test_explore14.mp4'))

Win/lose count 5.0/0. Average score (0.3333333333333333)
Win/lose count 7.0/1.0. Average score (0.7333333333333333)
Win/lose count 10.5/0. Average score (1.4333333333333333)
Win/lose count 12.5/0. Average score (2.2666666666666666)
Win/lose count 6.0/0. Average score (2.6666666666666665)
Win/lose count 1.5/0. Average score (2.7666666666666666)
Win/lose count 10.0/1.0. Average score (3.3666666666666667)
Win/lose count 1.0/0. Average score (3.433333333333333)
Win/lose count 9.5/0. Average score (4.066666666666666)
Win/lose count 1.0/0. Average score (4.133333333333334)
Win/lose count 4.5/0. Average score (4.433333333333334)
Win/lose count 9.0/0. Average score (5.033333333333333)
Win/lose count 6.5/0. Average score (5.466666666666667)
Win/lose count 3.0/0. Average score (5.666666666666667)
Win/lose count 11.0/0. Average score (6.4)
Final score: 6.4


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***